## D'NOVA YouTube Shorts Comments collection
- selenium을 통해 쇼츠 커멘트 수집
- 기본적으로 shorts url은 전부 가지고 있다고 가정

##### Test code

In [1]:
import time
import random
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys


In [2]:

def random_sleep():
    """ 1~2초 사이의 랜덤한 대기 """
    time.sleep(random.uniform(1, 2))

def scroll_to_bottom(driver):
    """ 페이지를 끝까지 스크롤 다운하는 함수 """
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        # Scroll down to bottom
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        
        # 스크롤 다운 후 로딩 시간을 추가 (1~2초 사이의 랜덤 대기)
        time.sleep(random.uniform(0.5, 1))
        
        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

def get_youtube_comments(url):
    # uc driver로 브라우저 실행
    driver = uc.Chrome()
    driver.get(url)
    random_sleep()  # 페이지가 로드될 시간을 줍니다.
    
    # 페이지 끝까지 스크롤 다운
    scroll_to_bottom(driver)
    
    # 모든 댓글 수집
    comments_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
    comments = [comment.text for comment in comments_elements]
    random_sleep()  # 데이터 수집 중간에도 랜덤 대기
    
    # 댓글 작성 날짜 수집
    date_elements = driver.find_elements(By.XPATH, '//*[@id="published-time-text"]/a')
    dates = [date.text for date in date_elements]
    random_sleep()  # 데이터 수집 중간에도 랜덤 대기
    
    # 결과를 리스트의 튜플 형태로 반환
    comments_data = list(zip(comments, dates))
    
    # 드라이버 종료
    driver.quit()
    
    return comments_data


In [1]:
# 예시 URL
url = "https://www.youtube.com/watch?v=xEuH_5Ik92U"
comments_list = get_youtube_comments(url)

# 리스트와 튜플 형태의 데이터 출력
print(len(comments_list))

NameError: name 'get_youtube_comments' is not defined

In [6]:
# Test code
import undetected_chromedriver as uc
import time
import random
from selenium.webdriver.common.by import By  # <-- Missing import added here

# Define the youtube_transcript function
def youtube_transcript(driver):
    """Extract transcript text from YouTube Shorts video page."""
    
    try:
        # Step 1: Click the 'more' button
        more_button = driver.find_element(By.XPATH, '//*[@id="expand"]')
        more_button.click()
        time.sleep(random.uniform(1, 2))  # Random delay to ensure smooth loading

        # Step 2: Click the 'show transcript' button
        transcript_button = driver.find_element(By.XPATH, '//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
        transcript_button.click()
        time.sleep(random.uniform(1, 2))  # Random delay for loading transcript

        # Step 3: Collect all transcript text
        transcript_elements = driver.find_elements(By.XPATH, '//*[@id="body"]/ytd-transcript-segment-list-renderer')
        transcript_texts = [element.text for element in transcript_elements]
        
        # Combine all transcript texts into a single string
        transcript_data = " ".join(transcript_texts) if transcript_texts else "No transcript found"
        
        return transcript_data  # Return the combined transcript text

    except Exception as e:
        print(f"Error occurred while extracting transcript: {e}")
        return None

# Main function to run the transcript extraction example
def main():
    # YouTube Shorts URL to scrape
    url = "https://www.youtube.com/watch?v=xEuH_5Ik92U"

    # Initialize the driver
    driver = uc.Chrome()
    driver.get(url)
    driver.implicitly_wait(10)  # Wait for page to load

    # Call the youtube_transcript function to get the transcript text
    transcript_text = youtube_transcript(driver)
    
    # Print the transcript text
    if transcript_text:
        print("Transcript Text:")
        print(transcript_text)
    else:
        print("Transcript not available.")

    # Close the driver
    driver.quit()

# Run the main function
main()


Transcript Text:
0:00
스피드가 태국에서 라이브를 하다가 한
0:01
여성팬을 만났습니다 손하고 말을
0:03
안하는 여성팬 알고보니 각장의
0:11
팬이었는데 그렇게 헤지고 몇 시간 후
0:13
그를 기해
0:19
스피드 그리고 캄보디아로 이동해서
0:21
라이브를 하 난
0:25
[음악]
0:27
남성 뭔 보니다
0:31
[박수]
0:41
그리고 다시 읽어보는
0:49
스피드 지금까지 소리없이 스피드를
0:52
마나 열심히 챙겨는 알 수 있어서
0:53
감동이네요


#### YouTube Shorts Comments & Transcript 수집
- Top 100 or more의 쇼츠 데이터의 comments와 script 데이터는 selenium을 통해 직접 개발 진행
- 우회를 위해 vpn or cookie를 통해 진행

In [1]:
import sys
import pandas as pd
import time
import pickle
import os
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import random
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
import re

In [2]:
# Top shorts 데이터 기준으로 수집 진행
top_100_df = pd.read_excel("loaded_db_100.xlsx")

##### YouTube Shorts Transcript 함수

In [3]:
# Define the youtube_transcript function
def youtube_transcript(driver):
    """Extract transcript text from YouTube Shorts video page."""
    
    try:
        driver.implicitly_wait(15)  # Wait for page to load
        # Step 1: Click the 'more' button
        more_button = driver.find_element(By.XPATH, '//*[@id="expand"]')
        more_button.click()
        time.sleep(random.uniform(1, 2))  # Random delay to ensure smooth loading

        # Step 2: Click the 'show transcript' button
        transcript_button = driver.find_element(By.XPATH, '//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]')
        transcript_button.click()
        time.sleep(random.uniform(1, 2))  # Random delay for loading transcript

        # Step 3: Collect all transcript text
        transcript_elements = driver.find_elements(By.XPATH, '//*[@id="body"]/ytd-transcript-segment-list-renderer')
        transcript_texts = [element.text for element in transcript_elements]
        
        # Combine all transcript texts into a single string
        transcript_data = " ".join(transcript_texts) if transcript_texts else "No transcript found"
        transcript_data = process_transcript(transcript_data)
        return transcript_data  # Return the combined transcript text

    except Exception as e:
        print(f"Error occurred while extracting transcript: {e}")
        return None
    

def process_transcript(transcript_text):
    # Remove time markers (e.g., "0:00")
    transcript_text = re.sub(r'\d+:\d+\s*', '', transcript_text)
    
    # Split by newline or sentence-ending punctuation
    sentences = re.split(r'(?<=[.?!])\s+', transcript_text.strip())
    
    # Remove any empty entries and return as a list of sentences
    return [sentence for sentence in sentences if sentence]

In [8]:
# test

# Main function to run the transcript extraction example
def main():
    # YouTube Shorts URL to scrape
    # url = "https://www.youtube.com/watch?v=xEuH_5Ik92U"

    # Initialize the driver
    driver = uc.Chrome()
    # driver.get(url)
    # driver.implicitly_wait(15)  # Wait for page to load

    # Call the youtube_transcript function to get the transcript text
    # transcript_text = youtube_transcript(driver)
    data = process_youtube_comments_transcript(driver, top_100_df)
    # Print the transcript text
    if data:
        print("Transcript Text:")
        print(data)
    else:
        print("Transcript not available.")

    # Close the driver
    driver.quit()
    return data

# Run the main function
data = main()



Error occurred while extracting transcript: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="primary-button"]/ytd-button-renderer/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]"}
  (Session info: chrome=130.0.6723.59); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x003C38B3+24035]
	(No symbol) [0x0034BC44]
	(No symbol) [0x0022C2D3]
	(No symbol) [0x0026DC86]
	(No symbol) [0x0026DECB]
	(No symbol) [0x002AB9D2]
	(No symbol) [0x0028FED4]
	(No symbol) [0x002A953F]
	(No symbol) [0x0028FC26]
	(No symbol) [0x0026218C]
	(No symbol) [0x0026310D]
	GetHandleVerifier [0x006696D3+2800643]
	GetHandleVerifier [0x006C428E+3172286]
	GetHandleVerifier [0x006BCEA2+3142610]
	GetHandleVerifier [0x00466C60+692624]
	(No symbol) [0x00354C5D]
	(No symbol) [0x00351968]
	(No symbol) [0x00351B00]
	(No symbol) [0x00343FB0]
	BaseThrea

#### Comments scoll down 후 날짜와 수집
- Block 될 경우 Cookie를 사용해 login 후 수집 진행
- 최대한 함수로 구조화 진행
- 기존 scaraping code 활용
- loaded_df_100: 추출 데이터중 전체 조회수 Top100 or 필터링 데이터 조회수 Top 100

In [9]:
import pandas as pd
import time
import random
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import os

# 우회를 위해 random하게 시간 배정
def random_sleep():
    """1~3초 사이의 랜덤한 대기"""
    time.sleep(random.uniform(1, 3))

# comments 끝까지 수집을 위해 scroll down
def scroll_to_bottom(driver):
    """페이지를 끝까지 스크롤 다운하는 함수"""
    # Check the comment count
    comment_count_element = driver.find_element(By.XPATH, '//*[@id="count"]/yt-formatted-string/span[2]')
    comment_count = int(comment_count_element.text.replace(',', ''))  # Remove commas and convert to int
    
    if comment_count <= 20:
        print("Comment count is 20 or less, no scrolling down.")
        return  # Exit if the comment count is 20 or less

    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    
    while True:
        driver.find_element(By.TAG_NAME, "body").send_keys(Keys.END)
        random_sleep()
        
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# 완료되는 부분까지 데이터 저장
def save_progress(df, output_file):
    """현재 진행 상태를 엑셀 파일로 저장하는 함수"""
    try:
        # Load existing data, if any
        existing_df = pd.read_excel(output_file) if os.path.exists(output_file) else pd.DataFrame()
        
        # Append new data
        df = pd.concat([existing_df, df], ignore_index=True)
        
        # Remove duplicate rows if needed
        df.drop_duplicates(subset=['channel_name', 'shorts_url'], keep='last', inplace=True)
        
        # Save the updated DataFrame back to the file
        df.to_excel(output_file, index=False)
        print(f"Progress saved to {output_file}")
    except Exception as e:
        print(f"Error saving progress: {e}")

# URL 접속 후 comments와 transcript 수집 진행
def process_youtube_comments_transcript(driver, top_100):
    """Process YouTube video comments and save progress whenever an error occurs or the program ends."""
    all_transcripts = []
    
    for index, row in top_100.iterrows():
        random_sleep()
        title = None
        
        try:
            url = row['shorts_url']
            title = row['channel_name']
            driver.get(url)
            driver.implicitly_wait(10)
            
            # 수집할 script 및 comments
            transcript_data = youtube_transcript(driver)  # your youtube_transcript function should be defined
            random_sleep()
            # 여기서 comment num 확인 
            scroll_to_bottom(driver)
            
            comments_elements = driver.find_elements(By.XPATH, '//*[@id="content-text"]/span')
            comments = [comment.text for comment in comments_elements]
            random_sleep()
            
            date_elements = driver.find_elements(By.XPATH, '//*[@id="published-time-text"]/a')
            dates = [date.text for date in date_elements]
            random_sleep()
            
            comments_data = list(zip(comments, dates)) if comments else [(None, None)]
            
            # Append data to DataFrame
            top_100.at[index, 'shorts_comments_time'] = [comments_data]
            top_100.at[index, 'transcript_data'] = transcript_data
            
            # Collect transcripts for return
            all_transcripts.append(transcript_data)

        except Exception as err:
            print(f"Error processing {title or 'Unknown'} at row {index}: {err}")
            save_progress(top_100, "loaded_db_100.xlsx")
            raise
    
    save_progress(top_100, "loaded_db_100.xlsx")
    return all_transcripts, top_100


##### Cookie 추가 우회 코드

In [6]:
# Define the range for the random sleep time in seconds
min_sleep_time, max_sleep_time = 1, 3

# Function to create ChromeOptions and a new Chrome driver
def create_driver():
    """Create a new instance of Chrome driver with fresh ChromeOptions."""
    options = uc.ChromeOptions()
    options.headless = False
    options.add_experimental_option('prefs', {'intl.accept_languages': 'en'})
    
    driver = uc.Chrome(options=options, use_subprocess=True)
    driver.set_window_size(1000, 800)
    return driver

initial_path = ""
# 미리 생성해둔 cookie 데이터
cookie_files = [
    "cookies_list/cookies_hyunjun960214.pkl",
    "cookies_list/cookies_ytcollect1.pkl",
    "cookies_list/cookies_yttrendreport.pkl"
]
cookie_index = 0

# cookie 데이터를 통해 진행
def login(driver):
    """Login to YouTube using the provided driver and rotating cookie files."""
    global cookie_index
    cookies_file_path = initial_path + cookie_files[cookie_index]
    print(f"Using cookies from: {cookies_file_path}")

    if os.path.exists(cookies_file_path):
        with open(cookies_file_path, 'rb') as cookies_file:
            cookies = pickle.load(cookies_file)
            driver.get('https://www.youtube.com')
            driver.implicitly_wait(10)
            for cookie in cookies:
                driver.add_cookie(cookie)
            print("Cookies added")
            driver.refresh()
            driver.implicitly_wait(10)
    else:
        driver.get('https://www.youtube.com')
        driver.implicitly_wait(15)
        try:
            clk = driver.find_element(By.XPATH, "(//div[@class='yt-spec-touch-feedback-shape yt-spec-touch-feedback-shape--touch-response-inverse'])[2]").click()
            time.sleep(random.uniform(1, 3))
        except:
            pass
        try:
            driver.find_element(By.XPATH, "//*[@id='buttons']/ytd-button-renderer/yt-button-shape/a/yt-touch-feedback-shape/div").click()
            time.sleep(random.uniform(1, 3))
            email_field = driver.find_element(By.XPATH, "//*[@id='identifierId']")
            email_field.send_keys("yttrendreport")
            time.sleep(random.uniform(1, 3))
            email_field.send_keys(Keys.ENTER)
            time.sleep(random.uniform(1, 3))
            email_pass = driver.find_element(By.XPATH, "//*[@id='password']/div[1]/div/div[1]/input")
            email_pass.send_keys("hjl960214!!")
            time.sleep(1)
            email_pass.send_keys(Keys.ENTER)
            time.sleep(random.uniform(1, 3))
            driver.implicitly_wait(10)
        except:
            pass
        with open(cookies_file_path, 'wb') as cookies_file:
            pickle.dump(driver.get_cookies(), cookies_file)
        print("Cookies saved")

    cookie_index = (cookie_index + 1) % len(cookie_files)

# 함수 합쳐 수집 진행 코드
def execute_crawling(top_1oo):
    try:
        driver = create_driver()
        # login(driver=driver)
        # driver = uc.Chrome()
        # driver.get(url)
        process_youtube_comments_transcript(driver, top_1oo)
        print(f"Process Waiting...")
        time.sleep(random.uniform(1, 2))
    except Exception as e:
        print(f"Error occurred while processing : {e}. Restarting with a new cookie.")
        
        if driver:
            driver.quit()
        
        time.sleep(random.uniform(60, 100))  
        # driver = uc.Chrome()
        # driver.get(url)
        driver = create_driver()  
        # login(driver=driver)
        try:
            process_youtube_comments_transcript(driver, top_1oo)
        except Exception as retry_error:
            print(f"Failed again while processing : {retry_error}")

    finally:
        if driver:
            driver.quit()


#### Program 실행

In [7]:
# Execute the crawling process
execute_crawling(top_100_df)

Error occurred while extracting transcript: Message: element not interactable
  (Session info: chrome=130.0.6723.59)
Stacktrace:
	GetHandleVerifier [0x003C38B3+24035]
	(No symbol) [0x0034BC44]
	(No symbol) [0x0022C14D]
	(No symbol) [0x0026EC2F]
	(No symbol) [0x0026414D]
	(No symbol) [0x0028FE8C]
	(No symbol) [0x00263C04]
	(No symbol) [0x00290124]
	(No symbol) [0x002A953F]
	(No symbol) [0x0028FC26]
	(No symbol) [0x0026218C]
	(No symbol) [0x0026310D]
	GetHandleVerifier [0x006696D3+2800643]
	GetHandleVerifier [0x006C428E+3172286]
	GetHandleVerifier [0x006BCEA2+3142610]
	GetHandleVerifier [0x00466C60+692624]
	(No symbol) [0x00354C5D]
	(No symbol) [0x00351968]
	(No symbol) [0x00351B00]
	(No symbol) [0x00343FB0]
	BaseThreadInitThunk [0x770F00C9+25]
	RtlGetAppContainerNamedObjectPath [0x77DA7B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77DA7B1E+238]

Error occurred while extracting transcript: Message: element not interactable
  (Session info: chrome=130.0.6723.59)
Stacktrace:
	GetHandleVer

KeyboardInterrupt: 